In [1]:
import csv

smiles = []
with open('zinc250k.csv', 'r') as csv_file:
    reader = csv.DictReader(csv_file)
    for row in reader:
        smiles.append(row['smiles'])

In [2]:
with open('MoleculeSTM_editing_SMILES.txt', 'r') as file:
    smiles.extend(file.readlines())

In [3]:
len(smiles)

249655

In [4]:
import sys
import importlib

sys.path.insert(0, '../graph-transformer')
# sys.path.insert(0, '/data/code/Transformer-M')
transformerm_models = importlib.import_module("Transformer_M.models")
TransformerM = transformerm_models.TransformerM
data = importlib.import_module("Transformer_M.data")
preprocess_item = data.wrapper.preprocess_item
collator_3d = data.collator_3d

2023-05-15 23:49:16 | INFO | rdkit | Enabling RDKit 2022.09.5 jupyter extensions


In [5]:
import os
import time
import argparse
from distutils.util import strtobool
from mflow.models.hyperparams import Hyperparameters
from mflow.utils.model_utils import load_model, get_latent_vec
import torch

parser = argparse.ArgumentParser()
parser.add_argument("--input_text", type=str, default='This molecule is beautiful.')
parser.add_argument("--checkpoint_name", type=str, default='littlegin=graphclinit_bert=scibert_epoch=299-step=18300.ckpt')
parser.add_argument("--model_dir", type=str, default='./results')
parser.add_argument("--data_dir", type=str, default='../data')
parser.add_argument('--data_name', type=str, default='qm9', choices=['qm9', 'zinc250k'], help='dataset name')
# parser.add_argument('--molecule_file', type=str, default='qm9_relgcn_kekulized_ggnp.npz',
#                     help='path to molecule dataset')
parser.add_argument("--snapshot-path", "-snapshot", type=str, required=True)
parser.add_argument("--hyperparams-path", type=str, default='moflow-params.json', required=True)
parser.add_argument("--gpu", type=int, default=-1)
parser.add_argument("--batch-size", type=int, default=100)
parser.add_argument('--additive_transformations', type=strtobool, default='false',
                    help='apply only additive coupling layers')
parser.add_argument('--delta', type=float, default=0.1)
parser.add_argument('--n_experiments', type=int, default=1, help='number of times generation to be run')
parser.add_argument('--seed', type=int, default=123)
parser.add_argument('--debug', action='store_true', default=False)
parser.add_argument('--temperature', type=float, default=1.0,
                    help='temperature of the gaussian distribution')
# parser.add_argument('--draw_neighborhood', type=strtobool, default='true',
#                     help='if neighborhood of a molecule to be visualized')

parser.add_argument('--save_fig', type=strtobool, default='true')
parser.add_argument('--save_score', type=strtobool, default='true')
parser.add_argument('-r', '--reconstruct', action='store_true', default=False)
# parser.add_argument('-i', '--interpolation', action='store_true', default=False)
parser.add_argument('--int2point', action='store_true', default=False)
parser.add_argument('--intgrid', action='store_true', default=False)

parser.add_argument('--inter_times', type=int, default=5)

parser.add_argument('--correct_validity', type=strtobool, default='true',
                    help='if apply validity correction after the generation')
args = parser.parse_args("--model_dir results/zinc250k_512t2cnn_256gnn_512-64lin_10flow_19fold_convlu2_38af-1-1mask -snapshot model_snapshot_epoch_200 --gpu 0 --data_name zinc250k --hyperparams-path moflow-params.json --temperature 0.85 --batch-size 1 --n_experiments 5".split(" "))

start = time.time()
print("Start at Time: {}".format(time.ctime()))
# chainer.config.train = False
snapshot_path = os.path.join(args.model_dir, args.snapshot_path)
hyperparams_path = os.path.join(args.model_dir, args.hyperparams_path)
print("loading hyperparamaters from {}".format(hyperparams_path))
model_params = Hyperparameters(path=hyperparams_path)
model = load_model(snapshot_path, model_params, debug=True)
if len(model.ln_var) == 1:
    print('model.ln_var: {:.2f}'.format(model.ln_var.item()))
elif len(model.ln_var) == 2:
    print('model.ln_var[0]: {:.2f}, model.ln_var[1]: {:.2f}'.format(model.ln_var[0].item(), model.ln_var[1].item()))

if args.gpu >= 0:
    # device = args.gpu
    device = torch.device('cuda:'+str(args.gpu) if torch.cuda.is_available() else 'cpu')
else:
    device = torch.device('cpu')
model.to(device)
print(f'device = {device}')
model.eval()  # Set model for evaluation

Start at Time: Mon May 15 23:49:19 2023
loading hyperparamaters from results/zinc250k_512t2cnn_256gnn_512-64lin_10flow_19fold_convlu2_38af-1-1mask/moflow-params.json
loading snapshot: results/zinc250k_512t2cnn_256gnn_512-64lin_10flow_19fold_convlu2_38af-1-1mask/model_snapshot_epoch_200
Hyper-parameters:
--------------------  ------------------------------------------------------------------------------------------------
b_n_type              4
b_n_flow              10
b_n_block             1
b_n_squeeze           19
b_hidden_ch           [512, 512]
b_affine              True
b_conv_lu             2
a_n_node              38
a_n_type              10
a_hidden_gnn          [256]
a_hidden_lin          [512, 64]
a_n_flow              38
a_n_block             1
mask_row_size_list    [1]
mask_row_stride_list  [1]
a_affine              True
path                  results/zinc250k_512t2cnn_256gnn_512-64lin_10flow_19fold_convlu2_38af-1-1mask/moflow-params.json
learn_dist            1
seed         

MoFlow(
  (bond_model): Glow(
    (blocks): ModuleList(
      (0): Block(
        (flows): ModuleList(
          (0): Flow(
            (actnorm): ActNorm()
            (coupling): AffineCoupling(
              (layers): ModuleList(
                (0): Conv2d(722, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
                (1): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
                (2): Conv2d(512, 1444, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
              )
              (norms): ModuleList(
                (0): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                (2): BatchNorm2d(1444, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              )
            )
          )
          (1): Flow(
            (actnorm): ActNorm()
            (coupling): AffineCoupling(
     

In [6]:
from mflow.utils.model_utils import smiles_to_adj, rescale_adj

def smiles_to_moflow_rep(smiles):
    adj, atoms = smiles_to_adj(smiles, 'zinc250k')
    adj_normalized = rescale_adj(adj)
    with torch.no_grad():
        device = next(model.parameters()).device
        z0, _ = model(adj.to(device), atoms.to(device), adj_normalized.to(device))

    h, adj_h = z0
    # Flatten h and adj_h into 1D tensors
    h_flat = h.view(h.shape[0], -1)
    adj_h_flat = adj_h.view(adj_h.shape[0], -1)

    return torch.cat([h_flat, adj_h_flat], dim=1)

In [7]:
from torch import Tensor
from types import SimpleNamespace
from torch_geometric.data import Data
import numpy as np
from ogb.utils.features import atom_to_feature_vector, bond_to_feature_vector
from rdkit.Chem import AllChem
from rdkit import Chem


def mol_to_graph_data_obj_simple(mol):
    """
    Converts rdkit mol object to graph Data object required by the pytorch
    geometric package. NB: Uses simplified atom and bond features, and represent
    as indices
    :param mol: rdkit mol object
    :return: graph data object with the attributes: x, edge_index, edge_attr
    """
    # atoms
    mol = Chem.AddHs(mol)

    bad = False

    # rdDepictor.Compute2DCoords(mol)
    if AllChem.EmbedMolecule(mol) == -1:
        bad = True
    # AllChem.EmbedMolecule(mol)

    mol_try = Chem.Mol(mol)
    if not bad:
        try:
            AllChem.MMFFOptimizeMolecule(mol_try)
            mol = mol_try
        except ValueError:
            print("oops")
            pass

    mol = Chem.RemoveHs(mol)

    num_atom_features = 2   # atom type,  chirality tag
    atom_features_list = []
    for atom in mol.GetAtoms():
        # atom_feature = [allowable_features['possible_atomic_num_list'].index(
        #     atom.GetAtomicNum())] + [allowable_features[
        #     'possible_chirality_list'].index(atom.GetChiralTag())]
        # atom_features_list.append(atom_feature)
        atom_features_list.append(atom_to_feature_vector(atom))

    x = torch.tensor(np.array(atom_features_list), dtype=torch.long)

    num_bond_features = 3  # bond type, bond stereo, is_conjugated
    if len(mol.GetBonds()) > 0: # mol has bonds
        edges_list = []
        edge_features_list = []
        for bond in mol.GetBonds():
            i = bond.GetBeginAtomIdx()
            j = bond.GetEndAtomIdx()

            edge_feature = bond_to_feature_vector(bond)

            # add edges in both directions
            edges_list.append((i, j))
            edge_features_list.append(edge_feature)
            edges_list.append((j, i))
            edge_features_list.append(edge_feature)

        # data.edge_index: Graph connectivity in COO format with shape [2, num_edges]
        edge_index = np.array(edges_list, dtype = np.int64).T

        # data.edge_attr: Edge feature matrix with shape [num_edges, num_edge_features]
        edge_attr = np.array(edge_features_list, dtype = np.int64)

    else:   # mol has no bonds
        edge_index = np.empty((2, 0), dtype = np.int64)
        edge_attr = np.empty((0, num_bond_features), dtype = np.int64)

    # positions
    try:
        if not bad:
            positions = mol.GetConformer().GetPositions()
        else:
            num_atoms = mol.GetNumAtoms()
            positions = np.zeros((num_atoms, 3))
    except ValueError:
        return None
    # bonds
    # num_bond_features = 2   # bond type, bond direction
    # if len(mol.GetBonds()) > 0: # mol has bonds
    #     edges_list = []
    #     edge_features_list = []
    #     for bond in mol.GetBonds():
    #         i = bond.GetBeginAtomIdx()
    #         j = bond.GetEndAtomIdx()
    #         edge_feature = [allowable_features['possible_bonds'].index(
    #             bond.GetBondType())] + [allowable_features[
    #                                         'possible_bond_dirs'].index(
    #             bond.GetBondDir())]
    #         edges_list.append((i, j))
    #         edge_features_list.append(edge_feature)
    #         edges_list.append((j, i))
    #         edge_features_list.append(edge_feature)

    #     # data.edge_index: Graph connectivity in COO format with shape [2, num_edges]
    #     edge_index = torch.tensor(np.array(edges_list).T, dtype=torch.long)

    #     # data.edge_attr: Edge feature matrix with shape [num_edges, num_edge_features]
    #     edge_attr = torch.tensor(np.array(edge_features_list),
    #                              dtype=torch.long)
    # else:   # mol has no bonds
    #     edge_index = torch.empty((2, 0), dtype=torch.long)
    #     edge_attr = torch.empty((0, num_bond_features), dtype=torch.long)

    data = Data(x=x, edge_index=torch.from_numpy(edge_index).to(torch.int64), edge_attr=torch.from_numpy(edge_attr).to(torch.int64))

    data.__num_nodes__ = len(x)
    data.pos = positions

    return data

class AttrDict(dict):
    def __init__(self, *args, **kwargs):
        super(AttrDict, self).__init__(*args, **kwargs)
        self.__dict__ = self


def data_to_graph(data):
    new_graph = AttrDict()
    new_graph.update(data.to_dict())
    new_graph = preprocess_item(new_graph)
    return new_graph

args = SimpleNamespace(_name='transformer_m_base', act_dropout=0.3, activation_fn='gelu', add_3d=True,
                               add_prev_output_tokens=False, all_gather_list_size=16384, amp=False,
                               amp_batch_retries=2,
                               amp_init_scale=128, amp_scale_window=None, apply_init=True,
                               arch='transformer_m_base',
                               attention_dropout=0.3, azureml_logging=False, batch_size=256, batch_size_valid=256,
                               best_checkpoint_metric='loss', bf16=False, bpe=None, broadcast_buffers=False,
                               bucket_cap_mb=25, checkpoint_shard_count=1, checkpoint_suffix='', clip_norm=0.0,
                               combine_valid_subsets=None, cpu=False, cpu_offload=False,
                               criterion='graph_prediction',
                               curriculum=0, data_buffer_size=20, data_path='NOT-IN-USE', dataset_impl=None,
                               dataset_name='NOT-IN-USE', ddp_backend='legacy_ddp', ddp_comm_hook='none',
                               device_id=0, disable_validation=False, distributed_backend='nccl',
                               distributed_init_method=None, distributed_no_spawn=False, distributed_num_procs=2,
                               distributed_port=-1, distributed_rank=0, distributed_world_size=2, dropout=0.1,
                               droppath_prob=0.1, edge_type='multi_hop', ema_decay=0.9999, ema_fp32=False,
                               ema_seed_model=None, ema_start_update=0, ema_update_freq=1, empty_cache_freq=0,
                               encoder_attention_heads=32, encoder_embed_dim=768, encoder_ffn_embed_dim=768,
                               encoder_layers=12, encoder_learned_pos=True, encoder_normalize_before=True, eos=2,
                               fast_stat_sync=False, find_unused_parameters=False, finetune_from_model=None,
                               fix_batches_to_gpus=False, fixed_validation_seed=None, force_anneal=None, fp16=False,
                               fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0,
                               fp16_scale_window=None, fp32_reduce_scatter=False, gen_subset='test',
                               gradient_as_bucket_view=False, grouped_shuffling=False, heartbeat_timeout=-1,
                               ignore_unused_valid_subsets=False, init_token=None, keep_best_checkpoints=-1,
                               keep_interval_updates=-1, keep_interval_updates_pattern=-1, keep_last_epochs=-1,
                               load_checkpoint_on_all_dp_ranks=False, localsgd_frequency=3, log_file=None,
                               log_format=None, log_interval=100, lr=[0.25], lr_scheduler='fixed', lr_shrink=0.1,
                               max_epoch=0, max_positions=512, max_tokens=None, max_tokens_valid=None, max_update=0,
                               max_valid_steps=None, maximize_best_checkpoint_metric=False,
                               memory_efficient_bf16=False,
                               memory_efficient_fp16=False, metric='mae', min_loss_scale=0.0001,
                               mode_prob='0.2,0.2,0.6', model_parallel_size=1, multi_hop_max_dist=5, no_2d=False,
                               no_epoch_checkpoints=False, no_last_checkpoints=False, no_progress_bar=False,
                               no_reshard_after_forward=False, no_save=False, no_save_optimizer_state=False,
                               no_seed_provided=False, no_shuffle=False, no_token_positional_embeddings=False,
                               noise_scale=0.2, not_fsdp_flatten_parameters=False, nprocs_per_node=2,
                               num_3d_bias_kernel=128, num_atoms=4608, num_classes=1, num_edge_dis=128,
                               num_edges=1536,
                               num_in_degree=512, num_out_degree=512, num_segment=2, num_shards=1, num_spatial=512,
                               num_workers=16, on_cpu_convert_precision=False, optimizer=None,
                               optimizer_overrides='{}',
                               pad=1, patience=-1, pipeline_balance=None, pipeline_checkpoint='never',
                               pipeline_chunks=0, pipeline_decoder_balance=None, pipeline_decoder_devices=None,
                               pipeline_devices=None, pipeline_encoder_balance=None, pipeline_encoder_devices=None,
                               pipeline_model_parallel=False, plasma_path='/tmp/plasma',
                               pooler_activation_fn='tanh',
                               profile=False, quantization_config_path=None, required_batch_size_multiple=8,
                               required_seq_len_multiple=1, reset_dataloader=False, reset_logging=False,
                               reset_lr_scheduler=False, reset_meters=False, reset_optimizer=False,
                               restore_file='checkpoint_last.pt', sandwich_ln=False, save_dir='NOT-IN-USE',
                               save_interval=1, save_interval_updates=0, scoring='bleu', seed=1, sent_loss=False,
                               sentence_avg=False, sentence_class_num=2, separator_token=None, shard_id=0,
                               share_encoder_input_output_embed=False, shorten_data_split_list='',
                               shorten_method='none', simul_type=None, skip_invalid_size_inputs_valid_test=False,
                               slowmo_base_algorithm='localsgd', slowmo_momentum=None, split='valid',
                               stop_min_lr=-1.0,
                               stop_time_hours=0, store_ema=False, suppress_crashes=False, task='NOT-IN-USE',
                               tensorboard_logdir=None, threshold_loss_scale=None, tokenizer=None, tpu=False,
                               train_subset='train', unk=3, update_epoch_batch_itr=False, update_freq=[1],
                               update_ordered_indices_seed=False, use_bmuf=False, use_plasma_view=False,
                               use_sharded_state=False,
                               user_dir='NOT-IN-USE',
                               valid_subset='valid', validate_after_updates=0, validate_interval=1,
                               validate_interval_updates=0, wandb_project=None, warmup_updates=0,
                               write_checkpoints_asynchronously=False, zero_sharding='none')

transformer = TransformerM(args=args)
transformer = transformer.to('cuda')
transformer.eval()

def forward_through_graph_encoder(collated):
    global transformer
    inner_states, atom_output = transformer.molecule_encoder(
        collated,
        segment_labels=None,
        perturb=None,
        last_state_only=True
    )

    last_state = inner_states[0]
    molecule_embedding = last_state.permute(1, 0, 2).mean(dim=1)
    return molecule_embedding

def smiles_to_transformerm_rep(mol_smiles):
    data = mol_to_graph_data_obj_simple(Chem.MolFromSmiles(mol_smiles))
    graph = data_to_graph(data)
    graph.idx = 0

    graph.y = np.array([0.0])

    max_node = 512
    multi_hop_max_dist = 5
    spatial_pos_max = 1024

    for idx, val in graph.items():
        if isinstance(val, np.ndarray):
            graph[idx] = torch.from_numpy(val)

    collated_graph = collator_3d([graph], max_node=max_node, multi_hop_max_dist=multi_hop_max_dist,
                                  spatial_pos_max=spatial_pos_max)


    for idx, val in collated_graph.items():
        if hasattr(val, 'to'):
            collated_graph[idx] = val.to('cuda')

    return forward_through_graph_encoder(collated_graph)


In [8]:
m_rep = smiles_to_moflow_rep("CCCC")
m_rep, m_rep.shape

(tensor([[ 2.5210, -3.0424, -0.9128,  ..., -4.5585, -2.8697, -3.3236]],
        device='cuda:0'),
 torch.Size([1, 6156]))

In [9]:
tm_rep = smiles_to_transformerm_rep("CCCC")
tm_rep, tm_rep.shape

(tensor([[-1.0292e+00, -6.3661e-01,  6.8536e-02,  1.0596e+00,  2.7937e-01,
           4.8032e-02,  3.3868e-03,  5.4359e-01,  6.3216e-01,  5.0395e-01,
          -8.3358e-02,  6.1866e-02,  5.4672e-01, -7.8906e-01, -1.0624e+00,
          -9.9871e-02, -9.8324e-01, -4.6070e-02,  3.9625e-01,  1.7100e-01,
          -5.5828e-01, -1.1153e+00,  5.7789e-01,  1.1025e+00, -7.8656e-01,
           5.2291e-01, -1.0449e+00,  1.1638e+00, -2.1258e-01, -3.8182e-01,
          -3.6330e-01, -3.6772e-01, -5.5444e-02,  6.6471e-01,  8.6532e-01,
           1.4292e+00, -9.7485e-01, -6.2611e-01,  5.8219e-01,  6.5893e-01,
           7.5214e-01, -4.3363e-01,  8.5513e-01,  6.1927e-01, -6.9865e-01,
          -7.6682e-01, -3.5654e-01,  1.8407e+00, -1.1714e+00, -1.1942e+00,
           1.3130e+00, -1.4000e+00,  4.8004e-01,  1.0684e-01, -8.3134e-01,
           1.7291e+00,  9.0076e-01, -6.8361e-01, -2.9527e-01,  1.2322e-01,
           1.2950e+00,  2.0847e+00, -1.7756e+00, -4.0259e-01, -2.2111e+00,
          -5.3987e-01,  8

In [10]:
from torch.utils.data import Dataset


class EmbeddingsDataset(Dataset):
    def __init__(self, smiles):
        self.smiles = smiles

    def __len__(self):
        return len(self.smiles)

    def __getitem__(self, idx):
        mol_smiles = self.smiles[idx]
        return smiles_to_moflow_rep(mol_smiles).squeeze(0), smiles_to_transformerm_rep(mol_smiles).squeeze(0)

dataset = EmbeddingsDataset(smiles)

In [11]:
d = dataset[0]
d[0].shape, d[1].shape

(torch.Size([6156]), torch.Size([768]))

In [16]:
import torch
from torch import nn, optim

class ProjectionModel(nn.Module):
    def __init__(self):
        super(ProjectionModel, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(6156, 4096),
            nn.ReLU(),
            nn.Linear(4096, 2048),
            nn.ReLU(),
            nn.Linear(2048, 768)
        )

    def forward(self, x):
        return self.model(x)

project_model = ProjectionModel()

In [17]:
# Define a loss function
def custom_loss(y_pred, y_true):
    return (y_pred - y_true).pow(2).sum(-1).mean()

In [18]:
# Define an optimizer
optimizer = optim.Adam(project_model.parameters(), lr=0.001, weight_decay=1e-4)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Move model to the device
project_model = project_model.to(device)

In [ ]:
from tqdm import tqdm
from torch.utils.data import DataLoader

train_loader = DataLoader(dataset=dataset, batch_size=32, shuffle=True)

num_epochs = 200
pbar_total = tqdm(total=num_epochs, desc="Training", ncols=70)
for epoch in range(num_epochs):
    total_loss = 0
    pbar_epoch = tqdm(total=len(train_loader), desc=f"Epoch {epoch+1}", ncols=70, leave=False)

    for i, (moflow_rep, transformerm_rep) in enumerate(train_loader):
        moflow_rep = moflow_rep.to(device)
        transformerm_rep = transformerm_rep.to(device)

        optimizer.zero_grad()
        outputs = project_model(moflow_rep)

        loss = (outputs - transformerm_rep).pow(2).sum(-1).mean()
        loss.backward()

        optimizer.step()

        total_loss += loss.item()
        pbar_epoch.update(1)

    pbar_epoch.close()
    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}, Loss: {avg_loss}")
    torch.save(project_model.state_dict(), f'edit-projection-epoch{epoch+1}-loss{avg_loss:.3f}.bin')
    pbar_total.update(1)

pbar_total.close()




Epoch 1:   0%|                    | 4/7802 [00:34<17:48:42,  8.22s/it]